In [1]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset,DatasetDict
from transformers import default_data_collator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
import json

In [2]:
qa_cased_bert_model = AutoModelForQuestionAnswering.from_pretrained("dkleczek/bert-base-polish-cased-v1")
tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized

In [3]:
def to_squad_format(dataset):
    res = {
        "id": [],
        "question": [],
        "answers": [],
        "context": [],
        "title": []
    }
    for act in dataset["data"]:
        title = act["title"]
        for para in act["paragraphs"]:
            ctx = para["context"]
            for q in para["qas"]:
                q["context"] = ctx
                q["title"] = title
                answers = {"answer_start": [], "text": []}
                for an in q["answers"]:
                    answers["answer_start"].append(an["answer_start"])
                    answers["text"].append(an["text"])
                q["answers"] = answers
                for c in ["id", "question", "answers", "context", "title"]:
                    res[c].append(q[c])
    return Dataset.from_dict(res)
    
def read_dataset():
    dataset = {}
    for t in ["train", "dev", "test"]:
        with open(f"lqad-pl-1/lqad-pl-{t}.json") as f:
            dataset[t] = to_squad_format(json.loads(f.read()))
    return DatasetDict(dataset)

dataset = read_dataset()


In [4]:
dataset["train"][0]

{'id': '2007_adw_1',
 'question': 'Wybrany do Sejmu może być obywatel polski mający prawo wybierania, który: ',
 'answers': {'answer_start': [115],
  'text': ['najpóźniej w dniu wyborów kończy 21 lat']},
 'context': ' Rozdział IV SEJM I SENAT Art . 99 § 1 . Wybrany do Sejmu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 21 lat . § 2 . Wybrany do Senatu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 30 lat . § 3 . Wybraną do Sejmu lub do Senatu nie może być osoba skazana prawomocnym wyrokiem na karę pozbawienia wolności za przestępstwo umyślne ścigane z oskarżenia publicznego .',
 'title': 'Konstytucja Rzeczypospolitej Polskiej z dnia 2 kwietnia 1997 r. uchwalona przez Zgromadzenie Narodowe w dniu 2 kwietnia 1997 r., przyjęta przez Naród w referendum konstytucyjnym w dniu 25 maja 1997 r., podpisana przez Prezydenta Rzeczypospolitej Polskiej w dniu 16 lipca 1997 r'}

In [5]:
# src: https://huggingface.co/docs/transformers/custom_datasets#load-squad-dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
def finetune(model):
    data_collator = default_data_collator

    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["dev"],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()
    return model

In [8]:
# qa_herbert_model_tuned = finetune(qa_herbert_model)
# qa_herbert_model_tuned.save_pretrained("herbert-base-cased-legal-qa-tuned")

qa_herbert_model_tuned = AutoModelForQuestionAnswering.from_pretrained("herbert-base-cased-legal-qa-tuned", local_files_only=True)

In [9]:
# qa_cased_bert_model_tuned = finetune(qa_cased_bert_model)
# qa_cased_bert_model_tuned.save_pretrained("bert-base-polish-cased-v1-legal-qa-tuned")

qa_cased_bert_model_tuned_t = AutoModelForQuestionAnswering.from_pretrained("bert-base-polish-cased-v1-legal-qa-tuned", local_files_only=True)

In [32]:
def evaluate(predictions, test_y):
    y = predictions
    print(predictions, test_y)
    accuracy = accuracy_score(y, test_y)
    precision = precision_score(y, test_y)
    recall = recall_score(y, test_y)
    f1 = f1_score(y, test_y)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return evaluate(pred, labels)

def evaluate_transformers(model):
    trainer = Trainer(model=model,
                      tokenizer=tokenizer)

    return trainer.predict(tokenized_dataset["test"])

In [34]:
predictions = evaluate_transformers(qa_herbert_model_tuned)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 463
  Batch size = 8


In [36]:
predictions

PredictionOutput(predictions=(array([[-7.4329634, -7.368832 , -6.617597 , ..., -7.5947294, -7.752664 ,
        -7.778024 ],
       [-7.4106345, -7.181142 , -6.629648 , ..., -7.7880297, -7.7957063,
        -7.798563 ],
       [-7.404494 , -7.2709823, -6.6942706, ..., -7.7840433, -7.820159 ,
        -7.8105807],
       ...,
       [-7.421554 , -7.0127587, -6.7157335, ..., -7.8349066, -7.8224626,
        -7.821253 ],
       [-7.3961077, -7.0576873, -6.702667 , ..., -7.8544455, -7.837531 ,
        -7.824364 ],
       [-7.6010027, -7.3299994, -6.6196427, ..., -7.9354043, -7.9293175,
        -7.9480586]], dtype=float32), array([[-7.1161537, -7.0751667, -6.731268 , ..., -7.2099695, -7.2679033,
        -7.239803 ],
       [-7.2239404, -7.1294746, -6.9316454, ..., -7.3927536, -7.386673 ,
        -7.373506 ],
       [-7.234407 , -7.2176886, -6.839652 , ..., -7.4629517, -7.448105 ,
        -7.4722033],
       ...,
       [-7.2303357, -7.15343  , -6.945491 , ..., -7.3965178, -7.3968616,
        -7

In [39]:
tokenized_dataset["test"]

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 463
})